In [2]:
import logging
import logging.config
import os
import json
# import ConfigParser
import sys
import re

from simpleMenu import Menu
import configuration
from gdrive.auth import getCredentials
from gdrive.gdrive import googledrive, GDriveError

from humanfriendly import prompts

# get the current working directory
# When launching a .command from the OS X Finder, the working directory is typically ~/; this is problematic
# for locating resource files
# I don't love this hack, but it works.
try:
    __file__
    cwd = os.path.dirname(__file__)+'/'
except NameError as e:
    cwd = os.getcwd()

In [3]:
from consolemenu import *
from consolemenu.format import *
from consolemenu.items import *

In [4]:
def setup_logging(
    default_path='logging.json',
    default_level=logging.INFO,
    env_key='LOG_CFG'
):
    """Setup logging configuration

    """
    path = default_path
    value = os.getenv(env_key, None)
    if value:
        path = value
    if os.path.exists(path):
        with open(path, 'rt') as f:
            config = json.load(f)
        logging.config.dictConfig(config)
        logging.getLogger().setLevel(default_level)
    else:
        logging.basicConfig(level=default_level)

In [5]:
def fileSearch(path = None, search = None):
        '''search for a file name string in a given path'''
        if not search:
            search = ''
        logger = logging.getLogger(__name__)
        logger.debug('searching path: {0} for \"{1}\"'.format(path, search))
        try:
            searchPath = os.path.expanduser(path)
        except AttributeError as e:
            logger.error('Error: no path specified')
            searchPath = ''
        mySearch = '*'+search+'*'
        try:
            allFiles = os.listdir(searchPath)
        except OSError as e:
            logger.error(e)
            return(None)
        
        # list comprehension search with regex
        #http://www.cademuir.eu/blog/2011/10/20/python-searching-for-a-string-within-a-list-list-comprehension/
        regex = re.compile('.*('+search+').*')
        return([m.group(0) for l in allFiles for m in [regex.search(l)] if m])

In [6]:
def getConfiguration(cfgfile):
    # required configuraiton options
    # Section: {'option': 'default value'}
    cfgpath = os.path.dirname(cfgfile)
    config_required = {
        'Main': {'credentials': os.path.join(cfgpath, 'credentials/'), 
                 'teamdriveID': '',
                 'teamdrivename': '',
                 'folderID': '',
                 'foldername': '',
                 },
        }

    config = configuration.get_config(cfgfile)

    update_config = False

    for section, values in config_required.items():
        if not config.has_section(section):
            logger.warn('section: {} not found in {}'.format(section, cfgfile))
            logger.debug('adding section {}'.format(section))
            config.add_section(section)
            update_config = True
        for option, value in values.items():
            if not config.has_option(section, option):
                logger.warn('option: {} not found in {}'.format(option, cfgfile))
                logger.debug('adding option {}: {}'.format(option, value))

                config.set(section, option, value)
                update_config = True


    # for section, options in config_required.items():

    if update_config:
        try:
            logger.info('updating configuration file at: {}'.format(cfgfile))
            configuration.create_config(cfgfile, config)
        except Exception as e:
            logger.error(e)
            
    return(config)

In [7]:
def getTeamDrive(myDrive):
    '''
    menu driven interaction for choosing a writeable team drive
    '''
    #available and writeable team drives
    logger = logging.getLogger(__name__)
    try:
        teamdrives_all = myDrive.listTeamDrives()
    except Exception as e:
        logger.error(e)
    
    if len(teamdrives_all) < 1:
        logger.warn('no writeable team drives found for this user')
        return(None)
    
    teamdrives_writeable = {}
    for drive in teamdrives_all:
        if drive['capabilities']['canAddChildren']:
            teamdrives_writeable[drive['name']] = drive
    
    teamdrive = None
    
    try:
        teamdrive_name = prompts.prompt_for_choice(choices=teamdrives_writeable, default=None)
        teamdrive=teamdrives_writeable[teamdrive_name]
    except prompts.TooManyInvalidReplies as e:
        logger.warning('user failed to make a choice')
    
    return(teamdrive)

In [83]:
def getPortfolioFolder(myDrive):
    '''
    menu driven interaction to locate and choose portfolio folder on team drive
    
    returns:
        folder (dictionary) - google drive object properties dictionary
    '''
    logger = logging.getLogger(__name__)
    logger.info('starting search for portfolio folder in teamdrive: {}; {}'.format(teamdriveName, teamdriveID))
    
    foldersearch = ''
    attempt = 5
    while (len(foldersearch) <= 0) and (attempt > 0):
        attempt -= 1
        foldersearch = prompts.prompt_for_input(question='Please enter a portion of the portfolio folder name (case insensitive): ',
                                                default=None, strip=True)
        if len(foldersearch) <= 0:
            print 'Nothing entered. {} attempts remain.'.format(attempt)
            
        searchresult = myDrive.search(name=foldersearch, fuzzy=True, teamdrive = teamdriveID, mimeType='folder')
        if len(searchresult['files']) <= 0:
            print 'No folders contained "{}"'.format(foldersearch)
            print 'Please try entering only part of the folder name'
            foldersearch = ''
            
    
    
    if foldersearch <= 0:
        logger.warn('user entered 0 length string for folder search')
        return(None)
    
    
    foldernames = []
    for each in searchresult['files']:
        foldernames.append(each['name'])
    
    try:
        foldername = prompts.prompt_for_choice(choices=foldernames)
    except prompts.TooManyInvalidReplies as e:
        logger.warning('user failed to make a choice')
        return(None)
    
    return(searchresult['files'][foldernames.index(foldername)])
    

In [9]:
version = '00.00 - 18.08.13'
appName = 'portfolio_creator'

cfgfile = appName+'.ini'
cfgpath = os.path.join('~/.config/', appName)
cfgfile = os.path.expanduser(os.path.join(cfgpath, cfgfile))
studentFile = None
updateConfig = False


logger = logging.getLogger(__name__)
setup_logging(default_level=logging.DEBUG)
logging.info('===Starting {} Log==='.format(appName))

myConfig = getConfiguration(cfgfile)

####### set all config options 
if myConfig.has_option('Main', 'credentials'):
    credential_store = os.path.expanduser(myConfig.get('Main', 'credentials'))
else:
    credential_store = os.path.expanduser(os.path.join(cfgpath, 'credentials'))
    updateConfig = True

if myConfig.has_option('Main', 'teamdriveid'):
    teamdriveID = myConfig.get('Main', 'teamdriveid')
else:
    teamdriveID = None
    
if myConfig.has_option('Main', 'teamdrivename'):
    teamdriveName = myConfig.get('Main', 'teamdrivename')
else:
    teamdriveName = None
    
if myConfig.has_option('Main', 'folderid'):
    folderID = myConfig.get('Main', 'folderid')
else:
    folderID = None

if myConfig.has_option('Main', 'foldername'):
    folderName = myConfig.get('Main', 'foldername')
else:
    folderName = None

if myConfig.has_option('Main', 'gradefolders'):
    gradefolders = myConfig.get('Main', 'gradefolders')
else:
    gradefolders = None
    
if logging.getLogger().getEffectiveLevel() <= 10:
    config_dict = {}
    for section in myConfig.sections():
        config_dict[section] = {}
        for option in myConfig.options(section):
            config_dict[section][option] = myConfig.get(section, option)

    logging.debug('current configuration:')
    logging.debug('\n{}'.format(config_dict))

# google drive authorization
logging.info('checking google credentials')
#### Add instructions before this shoots user over to a web authorization 
print '\n'*3
print 'You *may* be directed to a google web page that asks you to authorize this applicaiton.'
print 'Please choose an ASH account and authorize this applicaiton.'
print 'When you are done, please close the newly created tab in your web browser and return to this window.'
raw_input("Press Enter to continue...\n\n\n")
credential_store = os.path.expanduser(myConfig.get('Main', 'credentials'))

# attempt to authorize using stored credentials or generate new credentials as needed
try:
    credentials = getCredentials(credential_store)
except Exception as e:
    logging.critical(e)

# configure google drive object
myDrive = googledrive(credentials)

if not teamdriveID:
    logging.warn('Team Drive ID missing')
    teamdrive = getTeamDrive(myDrive)
    try:
        teamdriveID = teamdrive['id']
        teamdriveName = teamdrive['name']
    except (KeyError, TypeError) as e:
        
        logging.error('no valid team drive information found: {}'.format(e))
        exit(0)
        
    myConfig.set('Main', 'teamdriveid', teamdriveID)
    myConfig.set('Main', 'teamdrivename', teamdriveName)
    updateConfig = True
    

    
if not folderID:
    logging.warn('Folder ID missing')
    

if updateConfig:
    configuration.create_config(cfgfile, myConfig)


2018-09-07 22:24:51,548: [INFO: root.<module>] ===Starting portfolio_creator Log===
2018-09-07 22:24:51,549: [DEBUG: configuration.get_config] reading configuration file at: /Users/aaronciuffo/.config/portfolio_creator/portfolio_creator.ini
2018-09-07 22:24:51,551: [DEBUG: root.<module>] current configuration:
2018-09-07 22:24:51,551: [DEBUG: root.<module>] 
{'Main': {'credentials': '~/.config/portfolio_creator/credentials/', 'folderid': '', 'teamdriveid': '', 'teamdrivename': '', 'foldername': ''}}
2018-09-07 22:24:51,552: [INFO: root.<module>] checking google credentials




You *may* be directed to a google web page that asks you to authorize this applicaiton.
Please choose an ASH account and authorize this applicaiton.
When you are done, please close the newly created tab in your web browser and return to this window.
Press Enter to continue...



2018-09-07 22:24:52,694: [DEBUG: gdrive.auth.getCredentials] preparing google drive credentials
2018-09-07 22:24:52,694: [DEBUG: root.ge

In [84]:
foo = getPortfolioFolder()

2018-09-07 23:06:29,358: [INFO: __main__.getPortfolioFolder] starting search for portfolio folder in teamdrive: IT Blabla; 0ACLfU8KeD_BHUk9PVA

 Please enter a portion of the portfolio folder name (case insensitive): 
Nothing entered. 4 attempts remain.
2018-09-07 23:06:30,239: [DEBUG: root.search] apicall: files().list(q=mimeType="application/vnd.google-apps.folder", orderBy=createdTime))
2018-09-07 23:06:30,244: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/files?orderBy=createdTime&includeTeamDriveItems=true&corpora=teamDrive&supportsTeamDrives=true&q=mimeType%3D%22application%2Fvnd.google-apps.folder%22&teamDriveId=0ACLfU8KeD_BHUk9PVA&alt=json



 Please enter a portion of the portfolio folder name (case insensitive): Por
2018-09-07 23:06:33,483: [DEBUG: root.search] apicall: files().list(q=name contains "Por" and mimeType="application/vnd.google-apps.folder", orderBy=createdTime))
2018-09-07 23:06:33,489: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/files?orderBy=createdTime&includeTeamDriveItems=true&corpora=teamDrive&supportsTeamDrives=true&q=name+contains+%22Por%22+and+mimeType%3D%22application%2Fvnd.google-apps.folder%22&teamDriveId=0ACLfU8KeD_BHUk9PVA&alt=json


2018-09-07 23:06:33,700: [DEBUG: humanfriendly.prompts.prompt_for_choice] Requesting interactive choice on terminal (options are u'Portfolios', u'Ports', u'Portable' and u'Port Wine') ..

  1. Portfolios
  2. Ports
  3. Portable
  4. Port Wine
 
 Enter your choice as a number or unique substring (Control-C aborts): 
2018-09-07 23:06:36,126: [DEBUG: humanfriendly.prompts.prompt_for_choice] Got empty reply (there's no default choice), retrying (1/10) ..



 Error: Invalid input (there's no default choice).



  1. Portfolios
  2. Ports
  3. Portable
  4. Port Wine
 
 Enter your choice as a number or unique substring (Control-C aborts): 
2018-09-07 23:06:36,470: [DEBUG: humanfriendly.prompts.prompt_for_choice] Got empty reply (there's no default choice), retrying (2/10) ..


...


 Error: Invalid input (there's no default choice).



  1. Portfolios
  2. Ports
  3. Portable
  4. Port Wine
 
 Enter your choice as a number or unique substring (Control-C aborts): 
2018-09-07 23:06:41,183: [DEBUG: humanfriendly.prompts.prompt_for_choice] Got empty reply (there's no default choice), retrying (9/10) ..



 Error: Invalid input (there's no default choice).



  1. Portfolios
  2. Ports
  3. Portable
  4. Port Wine
 
 Enter your choice as a number or unique substring (Control-C aborts): 
2018-09-07 23:06:41,729: [DEBUG: humanfriendly.prompts.prompt_for_choice] Got empty reply (there's no default choice), retrying (10/10) ..
2018-09-07 23:06:41,730: [WARNING: humanfriendly.prompts.retry_limit] Received too many invalid replies on interactive prompt, giving up! (tried 10 times)
2018-09-07 23:06:41,731: [WARNING: __main__.getPortfolioFolder] user failed to make a choice



 Error: Invalid input (there's no default choice).


In [72]:
bar = foo['files']

In [ ]:
bar.

In [12]:
mydrive.search()
help(myDrive.search)

Help on method search in module gdrive.gdrive:

search(self, name=None, trashed=None, mimeType=False, fuzzy=False, date=None, dopperator='>', parents=None, orderBy='createdTime', teamdrive=None, quiet=True) method of gdrive.gdrive.googledrive instance
    search for an item by name and other properties in google drive
    
    args:
        name (string): item name in google drive - required
        trashed (bool): item is not in trash - default None (not used)
        mimeType = (string): item is one of the known mime types (gdrive.mimeTypes) - default None
        fuzzy = (bool): substring search of names in drive
        date = (RFC3339 date string): modification time date string (YYYY-MM-DD)
        dopperator (date comparison opprator string): <, >, =, >=, <=  - default >
        parents = (string): google drive file id string
        orderBy = (comma separated string): order results assending by keys below - default createdTime:
                    'createdTime', 'folder', 'modif

In [48]:
myTeamDrive = getTeamDrive()

2018-09-07 22:08:37,116: [INFO: googleapiclient.discovery.method] URL being requested: GET https://www.googleapis.com/drive/v3/teamdrives?fields=teamDrives&alt=json
2018-09-07 22:08:37,396: [DEBUG: humanfriendly.prompts.prompt_for_choice] Requesting interactive choice on terminal (options are u'ASH GDPR ', u'ITM-ITCC Internal', u'ASH PowerSchool Learning Resources', u'IT Blabla', u'ASH Transportation', u'ASH Medical Care Plans' and u'IT Support Internal') ..

  1. ASH GDPR 
  2. ITM-ITCC Internal
  3. ASH PowerSchool Learning Resources
  4. IT Blabla
  5. ASH Transportation
  6. ASH Medical Care Plans
  7. IT Support Internal
 
 Enter your choice as a number or unique substring (Control-C aborts): 
2018-09-07 22:08:38,817: [DEBUG: humanfriendly.prompts.prompt_for_choice] Got empty reply (there's no default choice), retrying (1/10) ..



 Error: Invalid input (there's no default choice).



  1. ASH GDPR 
  2. ITM-ITCC Internal
  3. ASH PowerSchool Learning Resources
  4. IT Blabla
  5. ASH Transportation
  6. ASH Medical Care Plans
  7. IT Support Internal
 
 Enter your choice as a number or unique substring (Control-C aborts): 
2018-09-07 22:08:38,968: [DEBUG: humanfriendly.prompts.prompt_for_choice] Got empty reply (there's no default choice), retrying (2/10) ..



 Error: Invalid input (there's no default choice).



  1. ASH GDPR 
  2. ITM-ITCC Internal
  3. ASH PowerSchool Learning Resources
  4. IT Blabla
  5. ASH Transportation
  6. ASH Medical Care Plans
  7. IT Support Internal
 
 Enter your choice as a number or unique substring (Control-C aborts): 
2018-09-07 22:08:39,124: [DEBUG: humanfriendly.prompts.prompt_for_choice] Got empty reply (there's no default choice), retrying (3/10) ..


...


 Error: Invalid input (there's no default choice).



  1. ASH GDPR 
  2. ITM-ITCC Internal
  3. ASH PowerSchool Learning Resources
  4. IT Blabla
  5. ASH Transportation
  6. ASH Medical Care Plans
  7. IT Support Internal
 
 Enter your choice as a number or unique substring (Control-C aborts): 
2018-09-07 22:08:40,086: [DEBUG: humanfriendly.prompts.prompt_for_choice] Got empty reply (there's no default choice), retrying (9/10) ..



 Error: Invalid input (there's no default choice).



  1. ASH GDPR 
  2. ITM-ITCC Internal
  3. ASH PowerSchool Learning Resources
  4. IT Blabla
  5. ASH Transportation
  6. ASH Medical Care Plans
  7. IT Support Internal
 
 Enter your choice as a number or unique substring (Control-C aborts): 
2018-09-07 22:08:40,497: [DEBUG: humanfriendly.prompts.prompt_for_choice] Got empty reply (there's no default choice), retrying (10/10) ..
2018-09-07 22:08:40,498: [WARNING: humanfriendly.prompts.retry_limit] Received too many invalid replies on interactive prompt, giving up! (tried 10 times)
2018-09-07 22:08:40,499: [WARNING: __main__.getTeamDrive] user failed to make a choice after



 Error: Invalid input (there's no default choice).


In [31]:
teamdrive_name = prompts.prompt_for_choice(choices=teamdrives_writeable, default=None, )
print teamdrives_writeable[teamdrive_name]

2018-09-07 21:56:54,720: [DEBUG: humanfriendly.prompts.prompt_for_choice] Requesting interactive choice on terminal (options are u'ASH GDPR ', u'ITM-ITCC Internal', u'ASH PowerSchool Learning Resources', u'IT Blabla', u'ASH Transportation', u'ASH Medical Care Plans' and u'IT Support Internal') ..

  1. ASH GDPR 
  2. ITM-ITCC Internal
  3. ASH PowerSchool Learning Resources
  4. IT Blabla
  5. ASH Transportation
  6. ASH Medical Care Plans
  7. IT Support Internal
 
 Enter your choice as a number or unique substring (Control-C aborts): 4
2018-09-07 21:56:58,187: [DEBUG: humanfriendly.prompts.prompt_for_choice] Option (u'IT Blabla') selected by numeric reply (4).
{u'restrictions': {u'adminManagedRestrictions': False, u'copyRequiresWriterPermission': False, u'domainUsersOnly': False, u'teamMembersOnly': False}, u'kind': u'drive#teamDrive', u'backgroundImageLink': u'https://lh3.googleusercontent.com/proxy/UzcLkepaZHw5GeAzjagJdp7-2DGaBMcj-9TU-X3o4PD1x_kQByZO_r-RDmBxS1iLzsq_MDEq4U_3w_qpTY_6

In [30]:
prompts.warning()

TypeError: warning() takes at least 1 argument (0 given)

In [ ]:
def xgetTeamDrive(myDrive):
    '''
    menu interaction to find appropriate teamdrive ID for storing portfolios
    '''
    logger = logging.getLogger(__name__)
    myMenu = Menu()
    teamDrives = {}
    logging.debug('fetching list of team drives')
    for each in myDrive.listTeamDrives():
        try:
            teamDrives[each['name']] = each['id']
            myMenu.addItem(each['name'])
        except KeyError as e:
            logger.warn('KeyError: {}'.format(e))
            
    myMenu.sortMenu()
    td_name = myMenu.loopChoice(optional=True, message='Choose Team Drive for Portfolios')
    if td_name.upper() == 'Q':
        return('Q', 'Q')
    else:
        return([td_name, teamDrives[td_name]])

In [ ]:
def xgetPortfolioFolder(myDrive, teamDriveID):
    # ask user for partial folder name
        # check for non 0 input
    # if results are returned ask user to choose from list, or search again
    # if no results are returned, prompt user for new search terms
    # return the name and id of the folder
    
    logger = logging.getLogger(__name__)
    logger.info('starting search for portfolio folder in teamdrive ID: {}'.format(teamdriveID))
    print 'Locating Portfolio Folder in Team Drive'
    
    search = True
    attempts = 3
    
    quitMenu = Menu(items=['Yes', 'No'])
    
    while search:
        logger.debug('remaining attempts: {}'.format(attempts))
        attempts -=1
        foldersearch = ''
        folderID = ''
        folderName = ''
        folderlist = {}
        myMenu = Menu()
        
        # get a partial file name to search for
        while len(foldersearch) <= 0:
            foldersearch = raw_input('Enter part of the portfolio folder name: ')
            if len(foldersearch) <=0:
                print 'Please enter at least one character.'
                
        # fetch results from google drive
        logger.debug('searching for folder name: {}'.format(foldersearch))
        searchresult = myDrive.search(name=foldersearch, fuzzy=True, teamdrive = teamdriveID, mimeType='folder')
        try:
            rawfilelist = searchresult['files']
        except KeyError as e:
            rawfilelist = {}
            logger.warn(e)
    
        if len(rawfilelist) < 1:
            print 'No matching folders were found. Please check your spelling and try a new search.'
            logger.debug('no matching folders found')
        else:
            for each_folder in rawfilelist:
                folderlist[each_folder['name']] = each_folder['id']
                myMenu.addItem(each_folder['name'])
                
            myMenu.sortMenu()
            folderName = myMenu.loopChoice()
            folderID = folderlist[folderName]
            if len(folderID) > 0:
                logger.debug('exiting folder choice loop')
                break
        
            
            
        # ask to break out of loop after attempts reaches 0
        if attempts <= 0:
            quit = quitMenu.loopChoice(message='It looks like you\'re having trouble. Would you like to try again?')
            if quit == 'No':
                search = False
            else:
                attempts = 3
        
    
    
    
    return(folderName, folderID)

In [ ]:
def getStudentFile():
    '''
    menu interaction to choose appropriate student_export.text file to import
    Returns:
        studentFile (string): path to student_export.text file
    '''
    logger = logging.getLogger(__name__)
    items = ['Desktop', 'Downloads', 'Documents']
    fileMenu = Menu(name='Location to search for "student_export file',items=items)
    fileMenu.sortMenu()
    studentFolder = fileMenu.loopChoice(optional=True,
                                        message='Please specify the location of a PowerSchool "student_export" file')
    if studentFolder == 'Q':
        logger.debug('user quit')
        return('Q')
    
    searchPath = os.path.expanduser('~/'+studentFolder)
    fileList = fileSearch(searchPath, 'text')
    logger.debug('files found: {}'.format(len(fileList)))
    if fileList is None:
        print '{0} folder does not exist!'.format(searchPath)
        return(None)
    elif len(fileList) < 1:
            print '{0} folder does not contain any files that end with ".text"\n'.format(searchPath)
            return(None)
    
    fileMenu = Menu(name = 'located student_export files', items = fileList)
    fileMenu.sortMenu()
    
    myFile = fileMenu.loopChoice(optional=True, message='Please choose the appropriate student_export file to import')
    
    if myFile == 'Q':
        logger.debug('user quit')
        return('Q')
    
    return(os.path.join(searchPath, myFile))
    

In [ ]:
def askContinue(studentExport, myConfig={}):
    '''
    menu interaction ask if everything is correct and run the folder population
    '''
    logger = logging.getLogger(__name__)
    logger.debug('checking configuration with user')
    continueMenu = Menu(name='Continue', items=['Yes', 'No'] ) 
    choice = None
    # print the configuration here
    cfgDisplay = ['teamdrivename', 'foldername', 'gradefolders', 'studenexport']
    print 'These are the settings that will be used to create portfolio folders:'
    for option in cfgDisplay:
        try:
            print '{}: {}'.format(option, myConfig.get('Main', option))
        except Exception as e:
            logger.debug(e)
    print 'studentexport: {}\n'.format(studentExport)
        
    
    while not choice:
        choice = continueMenu.loopChoice(optional=True, message='Begin folder creation with the settings shown above?')

        if choice == 'No':
            changeMenu = Menu(items=cfgDisplay, name='Current Configuration')
            change = changeMenu.loopChoice(optional=True, message='Which option would you like to change?')
            if change == 'teamdrivename':
                
            choice = None


        if choice == 'Yes':
            return('Y')

        if choice == 'Q':
            return('Q')
    
    return(choice)
    

In [ ]:
askContinue('foo', myConfig)

In [ ]:
version = '00.00 - 18.08.13'
appName = 'portfolio_creator'

cfgfile = appName+'.ini'
cfgpath = os.path.join('~/.config/', appName)
cfgfile = os.path.expanduser(os.path.join(cfgpath, cfgfile))
studentFile = None
updateConfig = False


logger = logging.getLogger(__name__)
setup_logging(default_level=logging.DEBUG)
logging.info('===Starting {} Log==='.format(appName))

myConfig = getConfiguration(cfgfile)

####### set all config options 
if myConfig.has_option('Main', 'credentials'):
    credential_store = os.path.expanduser(myConfig.get('Main', 'credentials'))
else:
    credential_store = os.path.expanduser(os.path.join(cfgpath, 'credentials'))
    updateConfig = True

if myConfig.has_option('Main', 'teamdriveid'):
    teamdriveID = myConfig.get('Main', 'teamdriveid')
else:
    teamdriveID = None
    
if myConfig.has_option('Main', 'teamdrivename'):
    teamdriveName = myConfig.get('Main', 'teamdrivename')
else:
    teamdriveName = None
    
if myConfig.has_option('Main', 'folderid'):
    folderID = myConfig.get('Main', 'folderid')
else:
    folderID = None

if myConfig.has_option('Main', 'foldername'):
    folderName = myConfig.get('Main', 'foldername')
else:
    folderName = None

if myConfig.has_option('Main', 'gradefolders'):
    gradefolders = myConfig.get('Main', 'gradefolders')
else:
    gradefolders = None
    
if logging.getLogger().getEffectiveLevel() <= 10:
    config_dict = {}
    for section in myConfig.sections():
        config_dict[section] = {}
        for option in myConfig.options(section):
            config_dict[section][option] = myConfig.get(section, option)

    logging.debug('current configuration:')
    logging.debug('\n{}'.format(config_dict))

# print '\n'*10
print 'Welcome to the portfolio creator for Google Team Drive version {0}'.format(version)
print 'This program will create portfolio folders in Google Team Drive for students.'
print 'You will need a student_export.text file from PowerSchool with at least the following information:'
print '     ClassOf, FirstLast, Student_Number\n'
print 'The order of the CSV does not matter, but the headers must be on the very first line.'
# raw_input("Press Enter to continue...\n\n\n")

    
# google drive authorization
logging.info('checking google credentials')
#### Add instructions before this shoots user over to a web authorization 
print '\n'*3
print 'You *may* be directed to a google web page that asks you to authorize this applicaiton.'
print 'Please choose an ASH account and authorize this applicaiton.'
print 'When you are done, please close the newly created tab in your web browser and return to this window.'
raw_input("Press Enter to continue...\n\n\n")
credential_store = os.path.expanduser(myConfig.get('Main', 'credentials'))

# attempt to authorize using stored credentials or generate new credentials as needed
try:
    credentials = getCredentials(credential_store)
except Exception as e:
    logging.critical(e)

# configure google drive object
myDrive = googledrive(credentials)

if (not teamdriveID) or (len(teamdriveID)< 1):
    teamdriveName, teamdriveID = getTeamDrive(myDrive)
    if teamdriveID == 'Q':
        logging.debug('user exit')
        sys.exit(0)
    myConfig.set('Main', 'teamdriveid', teamdriveID)
    myConfig.set('Main', 'teamdrivename', teamdriveName)
    updateConfig = True


if not folderID or len(folderID) < 1:
    folderName, folderID = getPortfolioFolder(myDrive, teamdriveID)
    if len(folderID) < 1:
        logging.debug('user exit')
        print 'exiting'
    myConfig.set('Main', 'folderID', folderID)
    myConfig.set('Main', 'folderName', folderName)
    updateConfig = True


if not gradefolders or len(gradefolders) < 1:
    logger.debug('searching cwd for gradefolders')
    if os.path.isfile('./gradefolders.txt'):
        logger.debug('gradefolders found in cwd')
        gradefolders = './gradefolders.txt'
        myConfig.set('Main', 'gradefolders', gradefolders)
        updateConfig = True   
    # if a grade folder on the desktop exists, use this one
    elif os.path.isfile(os.path.expanduser('~/Desktop/gradefolders.txt')):
        logger.debug('found gradefolders on ~/Desktop, using')
        gradefolders = os.path.expanduser('~/Desktop/gradefolders.txt')
    else:
        print 'The gradefolders.txt file appears to be missing.'
        print 'gradefolders.txt should contain the name of one grade level per line'
        print 'staring with Preeschool through 12th grade in the format below:'
        print '\n00-Preeschool\n00-Transition Kindergarten\n00-zKindergarten...\n09-Grade\n10-Grade\n11-Grade\n12-Grade'
        print 'please create a gradefolders.txt file and place it on the desktop'
        print 'exiting'
        sys.exit(0)
        

while not studentFile:
    studentFile = getStudentFile()
    if studentFile == 'Q':
        print 'exiting'
        sys.exit(0)
logger.debug('student file chosen: {}'.format(studentFile))



    
if updateConfig:
    configuration.create_config(cfgfile, myConfig)


